In [2]:
import pandas as pd
import numpy as np
import sqlite3

# --- PART 1: PYTHON ETL PIPELINE ---
print("🚀 Starting ETL Process...")

# 1. Extraction: Load raw transaction logs
df = pd.read_csv('../data/raw_sales_data.csv')
print(f"Loaded {len(df)} transactions.")

# 2. Cleaning: Handle missing regional data using imputation
missing_count = df['Region'].isnull().sum()
if missing_count > 0:
    print(f"Detected {missing_count} missing region values. Filling with 'Unknown'.")
    df['Region'].fillna('Unknown', inplace=True)

# 3. Transformation: Calculate financial metrics
# Calculating Total Revenue and Gross Profit for downstream analysis
df['Total_Revenue'] = df['Quantity'] * df['Unit_Price']
df['Total_Cost'] = df['Quantity'] * df['Unit_Cost']
df['Gross_Profit'] = df['Total_Revenue'] - df['Total_Cost']

# Create a 'Month' column for time-series aggregation
df['Month'] = pd.to_datetime(df['Date']).dt.to_period('M').astype(str)

print("✅ ETL complete. Data transformed.")

# --- PART 2: SQL ANALYTICS LAYER ---
# We use an in-memory SQLite database to perform complex aggregations
conn = sqlite3.connect(':memory:')
df.to_sql('sales', conn, index=False, if_exists='replace')

print("📊 Executing SQL Analytics...")

# A) Revenue Growth Analysis (Window Functions)
# Using LAG() to compare current month's revenue vs. previous month
query_growth = """
SELECT 
    Month,
    SUM(Total_Revenue) as Revenue,
    LAG(SUM(Total_Revenue), 1) OVER (ORDER BY Month) as Prev_Month_Rev,
    ROUND(
        (SUM(Total_Revenue) - LAG(SUM(Total_Revenue), 1) OVER (ORDER BY Month)) / 
        LAG(SUM(Total_Revenue), 1) OVER (ORDER BY Month) * 100, 
    2) as Growth_Percentage
FROM sales
GROUP BY Month
"""
df_growth = pd.read_sql_query(query_growth, conn)
print("\n--- Monthly Revenue Growth (SQL Window Functions) ---")
print(df_growth.tail(5))

# B) Churn Rate Calculation
# Logic: Identify customers active in the previous month who are NOT active in the current month.
query_churn = """
WITH MonthlyUsers AS (
    SELECT DISTINCT Customer_ID, Month FROM sales
),
ChurnStats AS (
    SELECT 
        curr.Month,
        COUNT(DISTINCT curr.Customer_ID) as Current_Users,
        COUNT(DISTINCT prev.Customer_ID) as Previous_Users,
        -- Count Retained Users (Active in both Current and Previous month)
        COUNT(DISTINCT CASE WHEN curr.Customer_ID IS NOT NULL THEN prev.Customer_ID END) as Retained_Users
    FROM MonthlyUsers curr
    LEFT JOIN MonthlyUsers prev 
      ON curr.Customer_ID = prev.Customer_ID 
      AND date(curr.Month || '-01') = date(prev.Month || '-01', '+1 month')
    GROUP BY curr.Month
)
SELECT 
    Month,
    Previous_Users,
    Retained_Users,
    -- Churn Rate Formula: (Users Last Month - Retained Users) / Users Last Month
    ROUND(((Previous_Users - Retained_Users) * 1.0 / Previous_Users) * 100, 1) as Churn_Rate_Pct
FROM ChurnStats
WHERE Previous_Users > 0
ORDER BY Month
"""
df_churn = pd.read_sql_query(query_churn, conn)
print("\n--- Churn Rate Analysis (SQL Logic) ---")
print(df_churn.tail(5))

print("\n✅ Analytics complete. Database ready for visualization.")

🚀 Starting ETL Process...
Loaded 5000 transactions.
Detected 150 missing region values. Filling with 'Unknown'.
✅ ETL complete. Data transformed.
📊 Executing SQL Analytics...

--- Monthly Revenue Growth (SQL Window Functions) ---
      Month    Revenue  Prev_Month_Rev  Growth_Percentage
6   2023-08  2164500.0       1573050.0              37.60
7   2023-09  3014850.0       2164500.0              39.29
8   2023-10  3118350.0       3014850.0               3.43
9   2023-11  2712900.0       3118350.0             -13.00
10  2023-12  1405050.0       2712900.0             -48.21

--- Churn Rate Analysis (SQL Logic) ---
     Month  Previous_Users  Retained_Users  Churn_Rate_Pct
4  2023-08             253             253             0.0
5  2023-09             308             308             0.0
6  2023-10             360             360             0.0
7  2023-11             356             356             0.0
8  2023-12             250             250             0.0

✅ Analytics complete. Data

/var/folders/82/n2wrm9914vn4m3chfl1g9pd80000gn/T/ipykernel_9005/2791281525.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Region'].fillna('Unknown', inplace=True)
